In [ ]:
import torch 
from torch import nn,optim
import torchvision 
from torchvision import datasets,transforms
from torch.utils.data import DataLoader 
from torch.utils.tensorboard import SummaryWriter 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
### Discriminator has no fc layers
class Discriminator(nn.Module):

    def __init__(self, channels_img, features_d):
        super(Discriminator,self).__init__()

        # Input shape : N x channels_img x 64 x 64
        self.disc = nn.Sequential(
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1), # 32x32
            nn.LeakyReLU(0.2),

            self._block(features_d, features_d*2, 4, 2, 1), # 16x16
            self._block(features_d*2, features_d*4, 4, 2, 1), # 8x8
            self._block(features_d*4, features_d*8, 4, 2, 1), # 4x4

            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0) # 1x1
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):

        return nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self,x):
        return self.disc(x)

In [ ]:
class Generator(nn.Module):

    def __init__(self, z_dim, channels_img, features_g):
        super(Generator,self).__init__()

        self.gen = nn.Sequential(

            self._block(z_dim, features_g*16, 4, 1, 0), # 4x4
            self._block(features_g*16, features_g*8, 4, 2, 1), # 8x8
            self._block(features_g*8, features_g*4, 4, 2, 1), # 16x16
            self._block(features_g*4, features_g*2, 4, 2, 1), # 32x32

            nn.ConvTranspose2d(features_g*2, channels_img, kernel_size=4, stride=2, padding=1), # 64x64
            nn.Tanh()

        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):

        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self,x):
        return self.gen(x)

In [ ]:
def initialize_weights(model):
    for m in model.modules() :
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

In [ ]:
def test():
    N, in_channels, H, W = 8, 3, 64, 64
    z_dim = 100 
    x = torch.randn((N, in_channels, H, W))

    disc = Discriminator(in_channels, 8)
    initialize_weights(disc)
    assert disc(x).shape == (N, 1, 1, 1)

    gen = Generator(z_dim, in_channels, 8)
    initialize_weights(gen)

    z = torch.randn((N, z_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W)

In [ ]:
BATCH_SIZE = 64
LR = 5e-5
CHANNELS_IMG = 1
FEATURES_D = 64
Z_DIM = 100
FEATURES_G = 64
IMAGE_SIZE = 64
step = 0
CRITIC_ITERATIONS = 5
WEIGHT_CLIP = 0.01

In [ ]:
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

t = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)])
])

data = datasets.MNIST('./data', transform=t, download=True, train=True)
loader = DataLoader(dataset=data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
disc = Discriminator(CHANNELS_IMG, FEATURES_D)
disc.to(dev)
initialize_weights(disc)
opt_disc = optim.RMSprop(disc.parameters(), lr = LR)

In [ ]:
gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_G)
gen.to(dev)
initialize_weights(gen)
opt_gen = optim.RMSprop(gen.parameters(), lr = LR)

In [ ]:
fixed_noise = torch.randn((32, Z_DIM, 1, 1)).to(dev)

writer_real = SummaryWriter(log_dir='runs/real')
writer_fake = SummaryWriter(log_dir='runs/fake')

gen.train()
disc.train()

In [ ]:
EPOCHS = 100
for epoch in range(EPOCHS):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(dev)

        for _ in range(CRITIC_ITERATIONS):
            noise = torch.randn((BATCH_SIZE, Z_DIM, 1, 1)).to(dev)
            fake = gen(noise)

            disc_real = disc(real).reshape(-1)
            disc_fake = disc(fake).reshape(-1)
            loss_D = -(torch.mean(disc_real) - torch.mean(disc_fake))

            disc.zero_grad()
            loss_D.backward(retain_graph=True)
            opt_disc.step()

            for p in disc.parameters():
                p.data.clamp_(-WEIGHT_CLIP, WEIGHT_CLIP)

        # train gen
        output = disc(fake).reshape(-1)
        loss_G = -torch.mean(output)
        gen.zero_grad()
        loss_G.backward()
        opt_gen.step()

        if batch_idx % 20 == 0:
            print(
                f"Epoch [{epoch}/{EPOCHS}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {loss_D:.4f}, loss G: {loss_G:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

    torch.save(disc.state_dict(), f'/content/drive/MyDrive/DCGAN/disc{step}.pt')
    torch.save(gen.state_dict(), f'/content/drive/MyDrive/DCGAN/gen{step}.pt')